In [ ]:
import tensorflow as tf
import math
import numpy as np
import pickle
import gc

In [ ]:
def create_NN_II():
    #Group I inputs-- 10
    input_size_I = (10,)
    input_size_II = (7,)
    #input_size_III = (2,)

    input_layers = []
    output_layers = []

    input_layer_1 = tf.keras.layers.Input(shape = input_size_I, name='inputI')
    input_layers.append(input_layer_1)
    dense_layer_1_1 = tf.keras.layers.Dense(4, activation='relu', name='inputI_dense1')(input_layer_1)
    dense_layer_1_2 = tf.keras.layers.Dense(4, activation='relu', name='inputI_dense2')(dense_layer_1_1)
    output_layers.append(dense_layer_1_2)

    #Group II
    input_layer_2 = tf.keras.layers.Input(shape = input_size_II, name='inputII')
    input_layers.append(input_layer_2)
    dense_layer_2_1 = tf.keras.layers.Dense(4, activation='relu', name='inputII_dense1')(input_layer_2)
    dense_layer_2_2 = tf.keras.layers.Dense(4, activation='relu', name='inputII_dense2')(dense_layer_2_1)
    output_layers.append(dense_layer_2_2)
    concat_output = tf.keras.layers.concatenate(output_layers, name='concatenated_output')

    '''#Group III
    for i in range(20):
        input_layer = tf.keras.layers.Input(shape=input_size_III, name=f'input_III_{i}')
        input_layers.append(input_layer)
        dense_layer_1 = tf.keras.layers.Dense(4, activation='relu', name=f'denseIII_1_{i}')(input_layer)
        dense_layer_2 = tf.keras.layers.Dense(8, activation='relu', name=f'denseIII_2_{i}')(dense_layer_1)
        output_layers.append(dense_layer_2)

    concat_output = tf.keras.layers.concatenate(output_layers, name='concatenated_output')'''

    x1 = tf.keras.layers.Dense(1024, activation='relu', name='dense1cct')(concat_output)
    x2 = tf.keras.layers.Dense(1024, activation='relu', name='dense2cct')(x1)
    x3 = tf.keras.layers.Dense(1024, activation='relu', name='dense3cct')(x2)
    output_layer = tf.keras.layers.Dense(1, activation='sigmoid')(x3)

    model = tf.keras.models.Model(inputs=input_layers, outputs=output_layer)
    return model



en_mod_II = create_NN_II()
en_mod_II.summary()

Model: "model_2"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputI (InputLayer)            [(None, 10)]         0           []                               
                                                                                                  
 inputII (InputLayer)           [(None, 7)]          0           []                               
                                                                                                  
 inputI_dense1 (Dense)          (None, 4)            44          ['inputI[0][0]']                 
                                                                                                  
 inputII_dense1 (Dense)         (None, 4)            32          ['inputII[0][0]']                
                                                                                            

In [ ]:
en_mod_II.compile(optimizer='adam',loss="binary_crossentropy",metrics=['accuracy'])

In [ ]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10646493319673961615
xla_global_id: -1
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 14236516352
locality {
  bus_id: 1
  links {
  }
}
incarnation: 338822451611963146
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 3080 Ti Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6"
xla_global_id: 416903419
]


In [ ]:
output_filepath = "ton_iot_datablockI_1897.pkl"
with open(output_filepath, 'rb') as f:
    groupI_block = pickle.load(f)
groupI_block.shape

(500000, 10)

In [ ]:
groupI_train = groupI_block[0:350000,:]
groupI_valid = groupI_block[350000:425000,:]
groupI_test = groupI_block[425000:,:]
groupI_train.shape,groupI_valid.shape,groupI_test.shape

((350000, 10), (75000, 10), (75000, 10))

In [ ]:
np.min(groupI_train), np.max(groupI_train)

(0.0, 1.0)

In [ ]:
output_filepath = "GroupII_1897_tvals.pkl"

with open(output_filepath, 'rb') as f:
    groupII_block = pickle.load(f)
groupII_block.shape

(500000, 7)

In [ ]:
groupII_train = groupII_block[0:350000,:]
groupII_valid = groupII_block[350000:425000,:]
groupII_test = groupII_block[425000:,:]
groupII_train.shape,groupII_valid.shape,groupII_test.shape

((350000, 7), (75000, 7), (75000, 7))

In [ ]:
np.min(groupII_train), np.max(groupII_train)

(0.0, 1.0)

In [ ]:
labels_block = np.load('labels_seeded_1897.npy')
labels_block.shape

(500000,)

In [ ]:
labels_tr = labels_block[0:350000,]
labels_va = labels_block[350000:425000,]
labels_te = labels_block[425000:,]
labels_tr.shape,labels_va.shape,labels_te.shape

((350000,), (75000,), (75000,))

In [ ]:
#for en_mod_s
EPOCHS = 200
batchSize = 64
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5, restore_best_weights=True)
history = en_mod_II.fit(
    x=[groupI_train,groupII_train],
    y=labels_tr,
    batch_size=batchSize,
    epochs=EPOCHS,
    validation_data=([groupI_valid,groupII_valid], labels_va),
    callbacks=[early_stopping],
    verbose=1
)

Epoch 1/200
5469/5469 [==============================] - 47s 8ms/step - loss: 0.1589 - accuracy: 0.9421 - val_loss: 0.1233 - val_accuracy: 0.9513
Epoch 2/200
5469/5469 [==============================] - 46s 8ms/step - loss: 0.1176 - accuracy: 0.9556 - val_loss: 0.0990 - val_accuracy: 0.9611
Epoch 3/200
5469/5469 [==============================] - 45s 8ms/step - loss: 0.1085 - accuracy: 0.9595 - val_loss: 0.1020 - val_accuracy: 0.9644
Epoch 4/200
5469/5469 [==============================] - 45s 8ms/step - loss: 0.1058 - accuracy: 0.9598 - val_loss: 0.1029 - val_accuracy: 0.9650
Epoch 5/200
5469/5469 [==============================] - 50s 9ms/step - loss: 0.1026 - accuracy: 0.9617 - val_loss: 0.0899 - val_accuracy: 0.9665
Epoch 6/200
5469/5469 [==============================] - 45s 8ms/step - loss: 0.0961 - accuracy: 0.9647 - val_loss: 0.0907 - val_accuracy: 0.9681
Epoch 7/200
5469/5469 [==============================] - 45s 8ms/step - loss: 0.0870 - accuracy: 0.9680 - val_loss: 0.0970 -